In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import pickle

from collections import ChainMap
from keras.utils.np_utils import to_categorical
from keras.models import load_model
from keras import Sequential
from keras import optimizers
from keras.layers import Conv2D,MaxPooling2D,Flatten,Softmax,Activation,Dense
from keras.layers import Input, Dropout, Lambda, Dot, Dense
from keras.models import Model
from keras.regularizers import l2
from keras.callbacks import Callback,ModelCheckpoint

from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_curve,roc_auc_score,accuracy_score
from tensorflow import keras
import tensorflow as tf

In [ ]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False

In [ ]:
def nom(file):
    w = open('/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Diag_Normalized.txt', 'w', encoding='utf-8')
    f = open(file, 'r')
    list1 = f.readline().rstrip('\r\n').split('\t')
    w.write('\t'.join(list1) + '\n')
    list2 = f.readline().rstrip('\r\n').split('\t')
    w.write('\t'.join(list2) + '\n')
    list3 = f.readline().rstrip('\r\n').split('\t')
    w.write('\t'.join(list3) + '\n')
    list3 = list3[1:]
    dic={}
    for l in f:
        sp = l.rstrip('\r\n').split('\t')
        dic[sp[0]] = sp
    for key in dic:
        flist = []
        sex = ['Male', 'Female']
        for i, ft in enumerate(dic[key][1:]):
            if i == 0:
              flist.append(ft)
            if i == 1:
               flist.append(sex.index(ft))
            if i == 2:
                if is_number(ft):
                  flist.append(float(ft) / 37.2)
                else:
                  flist.append(1)
            if i == 3:
                if ft in ['No underlying disease']:
                  flist.append(0)
                else:
                  flist.append(1)
            if i > 3:
                if '-' in list3[i]:
                    max = float(list3[i].split(' ')[0].split('-')[1])
                    min = float(list3[i].split(' ')[0].split('-')[0])
                elif '<' in list3[i]:
                    max = float(list3[i].split(' ')[0].split('<')[1])
                    min = 0
                ft = ft.replace('>', '').replace('<', '')
                if is_number(ft):
                  flist.append((float(ft) - min) / (max - min))
                else:
                  flist.append(0.5)
        w.write(key  + '\t' + '\t'.join(str(e) for e in flist) + '\n')


In [ ]:
files = "/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/prenorm_diag.txt"

In [ ]:
nom(files)

In [ ]:
norm_diag_file = pd.read_csv("/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Diag_Normalized.txt", delimiter = "\t").drop([0,1]).reset_index(drop=True)

In [ ]:
norm_diag_file

In [ ]:
array = ['Patient 1', 'Patient 2', 'Patient 3', 'Patient 4', 'Patient 5', 'Patient 115', 'Patient 117', 'Patient 121', 'Patient 126']
X_list = norm_diag_file.loc[norm_diag_file ['Patient'].isin(array)]

In [ ]:
X = norm_diag_file.iloc[:,1:]
X

In [ ]:
X = X.values.astype(float)


In [ ]:
X.shape

(1521, 127)

************************

In [ ]:
dfy = pd.read_csv('/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/ppcleancovid.csv')

In [ ]:
dfy["SARS-CoV-2 nucleic acids"].unique()

In [ ]:
y = list(dfy["SARS-CoV-2 nucleic acids"])

In [ ]:
y_list = list()
for i in y:
  if i == "Positive":
    i = 1
    y_list.append(i)
  else:
    i = 0
    y_list.append(i)
y = y_list

array = ['Patient 1', 'Patient 2', 'Patient 3', 'Patient 4', 'Patient 5', 'Patient 115', 'Patient 117', 'Patient 121', 'Patient 126']
dfy.loc[dfy['Patient'].isin(array)]
y = [1, 1, 1, 1, 1, 0, 0, 0, 0]

In [ ]:
Skf=StratifiedKFold(n_splits=10,shuffle=True)

In [ ]:
for train, val in Skf.split(X, y):
    print("TRAIN:", train, "TEST:", val)

In [ ]:
X_train=X[train]
X_val=X[val]

y_train=to_categorical(np.array(y)[train])
y_val=to_categorical(np.array(y)[val])

In [ ]:
def DNN_Simple(shape):
    X_in = Input(shape=(shape,))
    H2 = Dense(128, activation='relu', kernel_regularizer=l2(5e-4))(X_in)
    H3 = Dropout(0.5)(H2)
    H4 = Dense(64, activation='relu', kernel_regularizer=l2(5e-4))(H3)
    H5 = Dropout(0.2)(H4)
    H6 = Dense(64, activation='relu', kernel_regularizer=l2(5e-4))(H5)
    H7 = Dropout(0.2)(H6)
    H8 = Dense(48, activation='relu', kernel_regularizer=l2(5e-4))(H7)
    H9 = Dropout(0.2)(H8)
    H10 = Dense(16, activation='relu', kernel_regularizer=l2(5e-4))(H9)
    H11 = Dropout(0.5)(H10)
    Y = Dense(2, activation='sigmoid')(H11)
    model = Model(inputs=X_in, outputs=Y)
    model.compile(optimizer=tf.optimizers.Adam(decay=0.01),loss='binary_crossentropy'
                  ,metrics=['accuracy'])
    return model

In [ ]:
shape  = X_train.shape[1]
model  = DNN_Simple(shape)
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=2000, batch_size=64, class_weight = None, validation_data=(X_val, y_val), verbose=1)


In [ ]:
model.save('/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/struc_class_model.h5')
np.save('/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/struc_class_history.npy',history.history)

In [ ]:
model2 = load_model('/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/struc_class_model.h5')

array = ['Patient 1', 'Patient 2', 'Patient 3', 'Patient 4', 'Patient 5', 'Patient 115', 'Patient 117', 'Patient 121', 'Patient 126']
CF_X = norm_diag_file.loc[norm_diag_file ['Patient'].isin(array)]
CF_X =CF_X.iloc[:,1:].values.astype(float)


patient_list = norm_diag_file.loc[norm_diag_file ['Patient'].isin(array)]
patient_list  = list(patient_list.iloc[:,0])

CF_y = model2.predict(CF_X)

In [ ]:
df_CT_Predit =pd.DataFrame({'Patient':patient_list,'0':CF_y[:,0],'1':CF_y[:,1]})
df_CT_Predit.to_csv(r'/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/struc_class_Predict.csv',sep='\t',index=None)

In [ ]:
import pandas as pd

df2 = pd.read_csv('/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/struc_class_Predict.csv')